In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import time
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [2]:
path = r'C:\Users\harkn\OneDrive\Desktop\Tasks\Machine Learning in Python\Achievement 2\Project Data'

In [3]:
df_weather = pd.read_csv(os.path.join(path, 'DATASET_weather_clean_1-10.csv'))
df_answers = pd.read_csv(os.path.join(path, 'DATASET_predictions.csv'))
df_weather.columns

Index(['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure',
       'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine',
       'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max',
       'BELGRADE_cloud_cover',
       ...
       'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity',
       'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=135)

In [4]:
df_answers = df_answers.drop(columns=['DATE'])
df_answers.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Turn X and answers from a df to arrays

X = np.array(df_weather)
y = np.array(df_answers)

In [6]:
X = X.reshape(-1,15,9)
X.shape

(22950, 15, 9)

In [7]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
y.shape

(22950,)

In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360,)
(4590, 15, 9) (4590,)


In [10]:
# Reshape 3D (n_samples, timesteps, features) to 2D (n_samples, timesteps * features)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(X_train_flat.shape)  # Should be (18360, 15 * 9)
print(X_test_flat.shape)   # Should be (4590, 15 * 9)

(18360, 135)
(4590, 135)


In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_flat, y_train)

# Make predictions
y_pred = clf.predict(X_test_flat)

# Evaluate accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9342047930283225


In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360,)
(4590, 15, 9) (4590,)


In [25]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

def cnn_model(neurons, kernel, activation, optimizer, learning_rate, layers1, layers2, normalization, dropout, dropout_rate):
    activationL = ['relu', 'sigmoid', 'tanh']
    optimizerL = ['Adam', 'SGD', 'RMSprop']  # Use a list instead of a dictionary

    # Convert integer index to actual values
    activation = activationL[int(activation)]
    optimizer_name = optimizerL[int(optimizer)]

    # Map optimizer name to the corresponding TensorFlow optimizer class
    optimizer_mapping = {
        'Adam': Adam(learning_rate=learning_rate),
        'SGD': SGD(learning_rate=learning_rate),
        'RMSprop': RMSprop(learning_rate=learning_rate)
    }
    optimizer = optimizer_mapping[optimizer_name]

    # Build the model
    model = Sequential()
    model.add(Conv1D(neurons, kernel_size=int(kernel), activation=activation, input_shape=(timesteps, input_dim)))
    
    if normalization > 0.5:
        model.add(BatchNormalization())
    for _ in range(int(layers1)):
        model.add(Dense(neurons, activation=activation))
    if dropout > 0.5:
        model.add(Dropout(dropout_rate))
    for _ in range(int(layers2)):
        model.add(Dense(neurons, activation=activation))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [26]:
def train_and_evaluate(neurons, kernel, activation, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    model = cnn_model(neurons, kernel, activation, optimizer, learning_rate, layers1, layers2, normalization, dropout, dropout_rate)
    
    # Early stopping
    es = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        callbacks=[es],
        verbose=0
    )
    
    # Evaluate on the validation data
    val_acc = max(history.history['val_accuracy'])
    return val_acc


In [27]:
from bayes_opt import BayesianOptimization

# Define the objective function for optimization
def bay_area(neurons, kernel, activation, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    return train_and_evaluate(
        neurons=int(neurons),
        kernel=int(kernel),
        activation=int(activation),
        optimizer=int(optimizer),
        learning_rate=learning_rate,
        batch_size=int(batch_size),
        epochs=int(epochs),
        layers1=int(layers1),
        layers2=int(layers2),
        normalization=normalization,
        dropout=dropout,
        dropout_rate=dropout_rate
    )

# Parameter bounds for Bayesian Optimization
params = {
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation': (0, 2),  # relu, sigmoid, tanh
    'optimizer': (0, 2),   # Adam, SGD, RMSprop
    'learning_rate': (0.001, 0.1),
    'batch_size': (16, 128),
    'epochs': (10, 50),
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0, 0.5)
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(f=bay_area, pbounds=params, random_state=42)
optimizer.maximize(init_points=5, n_iter=10)

print("Best Parameters:", optimizer.max)


|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Restoring model weights from the end of the best epoch: 14.
| 1         | 0.7682    | 0.7491    | 122.5     | 0.732     | 0.2993    | 16.24     | 1.312     | 1.116     | 2.732     | 0.06051   | 73.73     | 0.02058   | 1.94      |
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.
| 2         | 0.7426    | 1.665     | 39.78     | 0.1818    | 0.0917    | 22.17     | 2.05      | 1.864     | 1.582     | 0.06157   | 22.55     | 0.2921    | 0.7327    |
Restoring model weights from the end of the best epoch: 32.
| 3         | 0.8151    | 0.9121    | 103.9     | 0.1997    | 0.2571    | 33.7      | 1.093     | 2.215     | 1.341     | 0.00744   | 95.4      | 0.9656    | 1.617     |
Restoring model weights from the end of the best epoch: 12.
| 4         | 0.7884    | 0.6092    | 26.94     | 0.6842    | 0.2201    | 14.88     | 1.99      | 1.069     | 2.819     | 0.02662   | 69.63     | 0.3117    | 1.04      |
Epoch 33: early stopping
Restoring model weights from th

C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 20.
| 6         | 0.8333    | 1.446     | 36.57     | 0.9054    | 0.371     | 47.31     | 2.758     | 2.099     | 1.304     | 0.02292   | 28.16     | 0.2206    | 0.9528    |


...

Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
| 13        | 0.6476    | 1.328     | 37.83     | 0.1128    | 0.1052    | 45.91     | 2.408     | 1.987     | 1.374     | 0.01302   | 28.36     | 0.2581    | 1.412     |


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
| 14        | 0.6476    | 1.839     | 34.92     | 0.9143    | 0.3003    | 48.28     | 2.634     | 2.157     | 1.569     | 0.0867    | 28.58     | 0.002577  | 0.4777    |


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 8.
| 15        | 0.6678    | 1.996     | 36.59     | 0.1355    | 0.1831    | 46.17     | 2.298     | 1.897     | 2.965     | 0.05645   | 28.61     | 0.4423    | 0.7726    |
Best Parameters: {'target': 0.8466775417327881, 'params': {'activation': 1.0934205586865593, 'batch_size': 36.70369901885903, 'dropout': 0.9695846277645586, 'dropout_rate': 0.3875664116805573, 'epochs': 47.579957662567566, 'kernel': 2.789654700855298, 'layers1': 2.19579995762217, 'layers2': 2.8437484700462337, 'learning_rate': 0.00976075770314003, 'neurons': 27.638457617723066, 'normalization': 0.045227288910538066, 'optimizer': 0.6506606615265287}}


In [28]:
# Define the Bayesian Optimization process
nn_opt = BayesianOptimization(f=bay_area, pbounds=params, random_state=42)

# Run the optimization
nn_opt.maximize(init_points=5, n_iter=10)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Restoring model weights from the end of the best epoch: 13.
| 1         | 0.7527    | 0.7491    | 122.5     | 0.732     | 0.2993    | 16.24     | 1.312     | 1.116     | 2.732     | 0.06051   | 73.73     | 0.02058   | 1.94      |
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 13.
| 2         | 0.768     | 1.665     | 39.78     | 0.1818    | 0.0917    | 22.17     | 2.05      | 1.864     | 1.582     | 0.06157   | 22.55     | 0.2921    | 0.7327    |
Epoch 21: early stopping
Restoring model weights from the end of the best epoch: 16.
| 3         | 0.79      | 0.9121    | 103.9     | 0.1997    | 0.2571    | 33.7      | 1.09

C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 26: early stopping
Restoring model weights from the end of the best epoch: 21.
| 6         | 0.8162    | 1.446     | 36.57     | 0.9054    | 0.371     | 47.31     | 2.758     | 2.099     | 1.304     | 0.02292   | 28.16     | 0.2206    | 0.9528    |


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
| 7         | 0.6476    | 0.6562    | 36.56     | 0.3483    | 0.1981    | 47.58     | 2.241     | 2.705     | 1.738     | 0.09888   | 29.61     | 0.3678    | 0.5625    |


...

Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
| 13        | 0.6476    | 1.364     | 27.48     | 0.3141    | 0.3152    | 14.27     | 1.874     | 1.546     | 2.516     | 0.03246   | 70.18     | 0.4137    | 0.1557    |


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 13.
| 14        | 0.7666    | 1.206     | 39.88     | 0.09083   | 0.2001    | 28.24     | 2.854     | 2.514     | 1.396     | 0.03812   | 87.14     | 0.9164    | 1.207     |


C:\Users\harkn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 5.
| 15        | 0.7263    | 0.5092    | 125.2     | 0.3827    | 0.4947    | 33.8      | 2.34      | 2.975     | 2.22      | 0.0564    | 99.72     | 0.8315    | 0.06602   |


In [29]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'sigmoid',
 'batch_size': 37,
 'dropout': 0.9695846277645586,
 'dropout_rate': 0.3875664116805573,
 'epochs': 48,
 'kernel': 2.789654700855298,
 'layers1': 2,
 'layers2': 3,
 'learning_rate': 0.00976075770314003,
 'neurons': 28,
 'normalization': 0.045227288910538066,
 'optimizer': <keras.src.optimizers.sgd.SGD at 0x2aa4acd0cd0>}

In [30]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  # Rounded kernel size for Conv1D
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = Adadelta(learning_rate=0.7631771981307285)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_30 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_123 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_124 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_30 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_126 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [32]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [33]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (18360, 15, 9)
y_train_one_hot shape: (18360, 15)


In [34]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [35]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
40/40 - 14s - 359ms/step - accuracy: 0.6030 - loss: 1.3633
Epoch 2/47
40/40 - 4s - 90ms/step - accuracy: 0.6904 - loss: 0.9027
Epoch 3/47
40/40 - 3s - 72ms/step - accuracy: 0.7272 - loss: 0.8071
Epoch 4/47
40/40 - 3s - 72ms/step - accuracy: 0.7492 - loss: 0.7454
Epoch 5/47
40/40 - 3s - 78ms/step - accuracy: 0.7652 - loss: 0.6965
Epoch 6/47
40/40 - 3s - 83ms/step - accuracy: 0.7771 - loss: 0.6588
Epoch 7/47
40/40 - 3s - 70ms/step - accuracy: 0.7875 - loss: 0.6206
Epoch 8/47
40/40 - 2s - 54ms/step - accuracy: 0.8004 - loss: 0.5846
Epoch 9/47
40/40 - 3s - 68ms/step - accuracy: 0.8097 - loss: 0.5571
Epoch 10/47
40/40 - 3s - 74ms/step - accuracy: 0.8192 - loss: 0.5314
Epoch 11/47
40/40 - 3s - 64ms/step - accuracy: 0.8226 - loss: 0.5130
Epoch 12/47
40/40 - 3s - 81ms/step - accuracy: 0.8304 - loss: 0.4924
Epoch 13/47
40/40 - 3s - 67ms/step - accuracy: 0.8354 - loss: 0.4742
Epoch 14/47
40/40 - 3s - 75ms/step - accuracy: 0.8415 - loss: 0.4567
Epoch 15/47
40/40 - 3s - 63ms/step - accu

In [36]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [37]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [38]:
y_pred = model.predict(X_test)

144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


In [39]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_classes))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      2955
           1       0.87      0.87      0.87       879
           2       0.71      0.66      0.68       162
           3       0.75      0.47      0.58        64
           4       0.21      0.16      0.18        25
           5       0.59      0.72      0.65        67
           6       0.67      0.22      0.33         9
           7       0.70      0.50      0.58        46
           8       0.00      0.00      0.00         7
           9       0.86      0.76      0.81       360
          10       1.00      0.12      0.22         8
          11       0.67      0.50      0.57         4
          13       1.00      0.33      0.50         3
          14       0.00      0.00      0.00         1

    accuracy                           0.89      4590
   macro avg       0.64      0.45      0.49      4590
weighted avg       0.89      0.89      

C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        2846        49         5       1           5         4       0   
BELGRADE      112       766         0       0           0         0       0   
BUDAPEST       23        32       107       0           0         0       0   
DEBILT         17         5        12      30           0         0       0   
DUSSELDORF      6         3         3       4           4         5       0   
HEATHROW        8         2         2       3           3        48       0   
KASSEL          1         2         1       0           2         1       2   
LJUBLJANA       9         2         8       0           0         3       1   
MAASTRICHT      5         0         0       0           0         1       0   
MADRID         21        22        12       2           4        19       0   
MUNCHENB        6         0         0       0       

In [58]:
# creating a RF classifier
clf = RandomForestClassifier()

In [60]:
grid_space={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }

In [66]:
start = time.time()
grid = GridSearchCV(clf,param_grid=grid_space,cv=3,scoring='accuracy', verbose=3, n_jobs=-1)
model_grid = grid.fit(X_train, y_train)
print('Search took %s minutes' % ((time.time() - start)/60))

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


ValueError: 
All the 1296 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\harkn\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 97, in validate_parameter_constraints
    raise InvalidParameterError(
sklearn.utils._param_validation.InvalidParameterE

In [72]:
def create_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Adjust activation based on your problem (e.g., softmax for multiclass)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [74]:
rs_space={'max_depth':list(np.arange(10, 100, step=10)) + [None],
              'n_estimators':np.arange(10, 500, step=50),
              'max_features':randint(1,7),
              'criterion':['gini','entropy'],
              'min_samples_leaf':randint(1,4),
              'min_samples_split':np.arange(2, 10, step=2)
         }

In [76]:
start = time.time()
rf_random = RandomizedSearchCV(clf2, rs_space, n_iter=200, scoring='accuracy', verbose=3, n_jobs=-1, cv=3) #500
model_random = rf_random.fit(X_train, y_train)
print('Search took %s minutes' % ((time.time() - start)/60))

NameError: name 'clf2' is not defined